<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2022_GoogleColabs/blob/main/Week7/Conv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 2883587     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-03-02 11:33:09.499349030 +0000
Modify: 2022-03-02 11:33:09.390349037 +0000
Change: 2022-03-02 11:33:09.390349037 +0000
 Birth: -
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-v_uf55nc
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-v_uf55nc
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=652213f3a316cf885994245fb57bc91386c30d789c0dc6773e183e061a316df2
  Stored in directory: /tmp/pip-ephem-wheel-cache-6r98wlzz/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [5]:
%%cu
#include <stdio.h>


inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void convolution_1D_basic_kernel(float* N, float* M, float* P, 
                                            int Mask_Width, int Width)
{ 
  int i = blockIdx.x*blockDim.x+threadIdx.x; 
  float Pvalue=0; 
  int N_start_point = i-Mask_Width/2; 
  for (int j=0; j<Mask_Width; j++) 
  {
    if(N_start_point+j>=0 && N_start_point+j< Width) 
    { 
      Pvalue += N[N_start_point+j]*M[j]; 
    } 
  }
  P[i]=Pvalue; 
}
   
int main() { 
		float   *a, *k, *c=0; 
    const int N = 16;
    const int MASK_WIDTH = 5;
    const int threadsPerBlock = 4;

    float kernel[MASK_WIDTH]={3,4,5,4,3};

    // Allocate Unified Memory -- accessible from CPU or GPU
    checkCudaErr(cudaMallocManaged(&a, N*sizeof(float)), "cudaMallocManaged1");
    checkCudaErr(cudaMallocManaged(&c, N*sizeof(float)), "cudaMallocManaged1");
    checkCudaErr(cudaMallocManaged(&k, MASK_WIDTH*sizeof(float)), "cudaMallocManaged2");
     
    // fill in the memory with data
    for (int i=0; i<N; i++) {
        a[i] = i+1;
        c[i] = 0;
        if(i<MASK_WIDTH)
            k[i]=kernel[i];
    }  
 
    const int blocksPerGrid =  (N / threadsPerBlock);
 
    // Prefetch the data to the GPU
    int device = -1;
    cudaGetDevice(&device);
    cudaMemPrefetchAsync(a, N*sizeof(float), device, NULL);
    cudaMemPrefetchAsync(k, MASK_WIDTH*sizeof(float), device, NULL);

    convolution_1D_basic_kernel<<<blocksPerGrid,threadsPerBlock>>>(a, k, c, MASK_WIDTH, N);
    
    cudaDeviceSynchronize();
 
    //output the result
    puts("Conv1D Result: ");
    for(int i=0; i<N; ++i)
      printf("p[%d] = %3.3f\n", i, c[i]);

    // free memory on the gpu side
    checkCudaErr( cudaFree( a ) , "cudaFree1");
    checkCudaErr( cudaFree( k ) , "cudaFree2");
    checkCudaErr( cudaFree( c ) , "cudaFree3");
		checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

		return 0;
}

Result: 
p[0] = 22.000
p[1] = 38.000
p[2] = 57.000
p[3] = 76.000
p[4] = 95.000
p[5] = 114.000
p[6] = 133.000
p[7] = 152.000
p[8] = 171.000
p[9] = 190.000
p[10] = 209.000
p[11] = 228.000
p[12] = 247.000
p[13] = 266.000
p[14] = 234.000
p[15] = 182.000

